In [12]:
# Import required modules
from langchain_ollama import ChatOllama
from langchain_groq  import ChatGroq
from langchain.agents import create_agent, AgentState
import custom_tools

In [13]:
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

basic_model = ChatOllama(model="mistral:7b",base_url="http://localhost:11434",temperature=0.0)
advanced_model = ChatGroq(model="openai/gpt-oss-20b", temperature=0.0)

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""

    complex_keywords = ["analysis", "research", "detailed", "comprehensive", "complex", "strategy"]

    found = False
    for keyword in complex_keywords:
        for msg in request.state['messages']:
            if(keyword in str(msg).lower()):
                found = True
                break

    message_count = len(request.state["messages"])
    if found or message_count > 1:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    return handler(request.override(model=model))


## Passing runtime to tool. Runtime enable tools to have state. Code below. check-out the tool named "summarize_conversation" in custom_tools.py

In [14]:
# Create agent with dynamic model selection
tool_list = [custom_tools.search_internet, custom_tools.calculate, custom_tools.summarize_conversation]
agent = create_agent(
    model=basic_model,  # Default model
    tools=tool_list,
    middleware=[dynamic_model_selection]
)

print("Dynamic agent created successfully!")
print("This agent will automatically switch models based on conversation complexity")

result1 = agent.invoke({
    "messages": ["Explain differences between Agentic AI and Generative AI?", "What is capital of India?", "Return count of messages and tool calls using the tool"]
})

Dynamic agent created successfully!
This agent will automatically switch models based on conversation complexity


In [15]:
print(result1)

{'messages': [HumanMessage(content='Explain differences between Agentic AI and Generative AI?', additional_kwargs={}, response_metadata={}, id='39792b29-606e-44ee-8071-16f64b547d12'), HumanMessage(content='What is capital of India?', additional_kwargs={}, response_metadata={}, id='ef3a5ec8-73e1-473c-91ca-2caa612f655e'), HumanMessage(content='Return count of messages and tool calls using the tool', additional_kwargs={}, response_metadata={}, id='a6fba0b7-ec3b-4c2e-9045-d01ab5019d6b'), AIMessage(content='**Message count:** 3  \n**Tool calls:** 0', additional_kwargs={'reasoning_content': 'The user wants: "Return count of messages and tool calls using the tool". They want to use the tool to get the count? There\'s a tool "summarize_conversation" that can summarize conversation so far. But they want count of messages and tool calls. There\'s no direct tool for that. We can use summarize_conversation to get summary, but not count. We could manually count: messages: user asked 3 times? Let\'s

In [8]:
result1 = agent.invoke({
    "messages": ["Explain differences between Agentic AI and Generative AI?", "What is capital of India?"]
})

print(result1)

{'messages': [HumanMessage(content='Explain differences between Agentic AI and Generative AI?', additional_kwargs={}, response_metadata={}, id='31bf7d13-0234-400b-8cf7-e4cfae487870'), HumanMessage(content='What is capital of India?', additional_kwargs={}, response_metadata={}, id='1d3d730f-eff6-4f5a-ae0b-b6cde87c83f9'), AIMessage(content='**Agentic AI vs. Generative AI**\n\n| Feature | Agentic AI | Generative AI |\n|---------|------------|---------------|\n| **Primary Goal** | To act autonomously or semi‑autonomously to achieve a specific objective or set of objectives (e.g., scheduling, trading, game playing). | To produce new content that mimics or extends existing data (e.g., text, images, music, code). |\n| **Core Capability** | Decision‑making, planning, and execution in dynamic environments. | Creative synthesis, pattern generation, and style transfer. |\n| **Typical Architecture** | Often combines perception modules (e.g., vision, language) with a **policy** or **controller** th

In [10]:
print(result1["messages"][-1].content)


Here’s the breakdown from the conversation so far:

- **User messages:** 3  
- **AI responses:** 2  
- **Tool calls:** 1  

So, in total there have been **5 messages** exchanged, and **1 tool call** has been made.
